In [59]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Bidirectional
from keras import backend as K

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
all_data = pd.read_csv("./2022.csv")
all_data["Timestamp"]=pd.to_datetime(all_data["Timestamp"],format="%Y-%m-%d") 
all_data.index=all_data['Timestamp'] 
all_data

In [48]:
data = []

In [65]:
def generateX(a, n):
     x_train = []
     y_train = []
     for i in range(len(a)):
         x = a[i:(i + n)]
         if (i + n) < len(a):
             x_train.append(x)
             y_train.append(a[i + n])
         else:
             break
     return np.array(x_train), np.array(y_train)

In [66]:
def ts_train_test(all_data, time_steps, for_periods): 

    ts_train = all_data[:'2021'].iloc[:,0:1].values
    ts_test = all_data['2022':].iloc[:,0:1].values
    ts_train_len = len(ts_train)
    ts_test_len = len(ts_test)
    
    X_train = [] 
    y_train = [] 
    y_train_stacked = [] 
    for i in range(time_steps, ts_train_len - 1): 
        X_train.append(ts_train[i-time_steps:i,0])
        y_train.append(ts_train[i:i+for_periods,0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    

    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    

    inputs = pd.concat((all_data["hash_rate"][:'2021'], all_data["hash_rate"]['2022':]), axis=0).values
    inputs = inputs[len(inputs)-len(ts_test) - time_steps:]
    inputs = inputs.reshape(-1,1)
    
    X_test = []
    for i in range(time_steps, ts_test_len+ time_steps- for_periods):
        X_test.append(inputs[i-time_steps:i,0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
    
    return X_train, y_train, X_test 

X_train, y_train, X_test = ts_train_test(all_data,5,2)
X_train.shape[0], X_train.shape[1]

(32, 5)

In [67]:

X_train_see = pd.DataFrame(np.reshape(X_train, (X_train.shape[0], X_train.shape[1])))
y_train_see = pd.DataFrame(y_train)
pd.concat([X_train_see, y_train_see], axis = 1)

X_test_see = pd.DataFrame(np.reshape(X_test, (X_test.shape[0], X_test.shape[1])))
pd.DataFrame(X_test_see)

print("There are " + str(X_train.shape[0]) + " samples in the training data")
print("There are " + str(X_test.shape[0]) + " samples in the test data")

There are 32 samples in the training data
There are 10 samples in the test data


In [68]:
x= X_train_see
y= y_train_see

In [69]:
x, y = generateX(data, 38)
x = x.reshape(-1,10,1)
y = y.reshape(-1,1)


x_train = x[:32 :, :]
y_train = y[:32:, :]
x_test = x[32:, :, :]
y_test = y[32:, :]

In [ ]:
K.clear_session()    
xInput = Input(batch_shape=(None, x_train.shape[1], x_train.shape[2]))
xLstm_1 = LSTM(10, return_sequences = True)(xInput)
xLstm_2 = Bidirectional(LSTM(10))(xLstm_1)
xOutput = Dense(1)(xLstm_2)


model = Model(xInput, xOutput)
model.compile(loss='mse', optimizer='adam')


model.fit(x_train, y_train, epochs=500, batch_size=20,verbose=1)

y_hat = model.predict(x_test, batch_size=1)


a_axis = np.arange(0, len(y_train))
b_axis = np.arange(len(y_train), len(y_train) + len(y_hat))

plt.figure(figsize=(10,6))
plt.plot(a_axis, y_train.reshape(32,), 'o-')
plt.plot(b_axis, y_hat.reshape(10,), 'o-', color='red', label='Predicted')
plt.plot(b_axis, y_test.reshape(10,), 'o-', color='green', alpha=0.2, label='Actual')
plt.legend()
plt.show()



In [ ]:
def generateX(a, n):
     x_train = []
     y_train = []
     for i in range(len(a)):
         x = a[i:(i + n)]
         if (i + n) < len(a):
             x_train.append(x)
             y_train.append(a[i + n])
         else:
             break
     return np.array(x_train), np.array(y_train)


 x, y = generateX(data, 38)
 x = x.reshape(-1,10,1)
 y = y.reshape(-1,1)

 x_train = x[:70, :, :]
 y_train = y[:70:, :]
 x_test = x[70:, :, :]
 y_test = y[70:, :]


 K.clear_session()   
 xInput = Input(batch_shape=(None, x_train.shape[1], x_train.shape[2]))
 xLstm_1 = LSTM(10, return_sequences = True)(xInput)
 xLstm_2 = Bidirectional(LSTM(10))(xLstm_1)
 xOutput = Dense(1)(xLstm_2)


 model = Model(xInput, xOutput)
 model.compile(loss='mse', optimizer='adam')




 model.fit(x_train, y_train, epochs=500, batch_size=20,verbose=1)



 y_hat = model.predict(x_test, batch_size=1)

 a_axis = np.arange(0, len(y_train))
 b_axis = np.arange(len(y_train), len(y_train) + len(y_hat))

 plt.figure(figsize=(10,6))
 plt.plot(a_axis, y_train.reshape(70,), 'o-')
 plt.plot(b_axis, y_hat.reshape(20,), 'o-', color='red', label='Predicted')
 plt.plot(b_axis, y_test.reshape(20,), 'o-', color='green', alpha=0.2, label='Actual')
 plt.legend()
 plt.show()

In [ ]:
train = df.loc[:36, ['hash_rate']]
test = df.loc[36:, ['hash_rate']]
# Feature는 Unadjusted 한 개

ax = train.plot()
test.plot(ax=ax)
plt.legend(['train', 'test'])



In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()

train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)

train_sc

In [ ]:
train_sc_df = pd.DataFrame(train_sc, columns=['hash_rate'], index=train.index)
test_sc_df = pd.DataFrame(test_sc, columns=['hash_rate'], index=test.index)
train_sc_df.head()

In [ ]:
for s in range(1, 13):
    train_sc_df['shift_{}'.format(s)] = train_sc_df['hash_rate'].shift(s)
    test_sc_df['shift_{}'.format(s)] = test_sc_df['hash_rate'].shift(s)

train_sc_df.head(13)

In [ ]:
X_train = train_sc_df.dropna().drop('hash_rate', axis=1)
y_train = train_sc_df.dropna()[['hash_rate']]

X_test = test_sc_df.dropna().drop('hash_rate', axis=1)
y_test = test_sc_df.dropna()[['hash_rate']]

In [ ]:
X_train = X_train.values
X_test= X_test.values

y_train = y_train.values
y_test = y_test.values

In [ ]:
X_train_t = X_train.reshape(X_train.shape[0], 12, 1) 
X_test_t = X_test.reshape(X_test.shape[0], 12, 1)

In [ ]:
from keras.layers import LSTM 
from keras.models import Sequential 
from keras.layers import Dense 
import keras.backend as K 
from keras.callbacks import EarlyStopping 
K.clear_session() 
model = Sequential() 
model.add(LSTM(20, input_shape=(36, 1))) # (timestep, feature) 
model.add(Dense(1)) # output = 1 
model.compile(loss='mean_squared_error', optimizer='adam') 
model.summary()


In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(X_train_t, y_train, epochs=100,
          batch_size=30, verbose=1, callbacks=[early_stop])

In [ ]:
X_test_t

In [ ]:
y_pred = model.predict(X_test_t)
print(y_pred)

In [ ]:
y_pred = model.predict(X_test_t)
y_pred = sc.inverse_transform(y_pred)
y_pred_df = pd.DataFrame(y_pred, columns=['prediction'], index=test.index[:-12])

y_pred_df.plot(ax=ax)
plt.show()